In [352]:
import pandas as pd
import numpy as np

### Scrape the First Page of the Coffee Data

In [353]:
import requests
from bs4 import BeautifulSoup
source_code = requests.get('https://database.coffeeinstitute.org/coffees/arabica')
soup = BeautifulSoup(source_code.text, 'html5lib')

In [452]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
chromedriver = "/Applications/chromedriver" 
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get('https://database.coffeeinstitute.org/coffees/arabica')

In [355]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [356]:
list_of_dfs=soup.find('table', {'data-table':'coffees'})

In [357]:
# This is just grabbing one page, will need to grab the rest of the pages later.
list_of_dfs = pd.read_html(driver.page_source)
list_of_dfs[0]
all_coffee=pd.DataFrame(list_of_dfs[0])
all_coffee.drop(['Unnamed: 0'], axis = 1, inplace = True)
all_coffee['ID']=all_coffee['ID'].str.strip('#')
all_coffee.head()

,ID,Species,Country,Owner,Grade,ICP,Completed
0,512197,Arabica,Ethiopia,metad plc,90.58,metad,"April 4th, 2015"
1,398980,Arabica,Ethiopia,metad plc,89.92,metad,"April 4th, 2015"
2,411500,Arabica,Guatemala,Grounds for Health Admin,89.75,scaa,"May 31st, 2010"
3,632909,Arabica,Ethiopia,Yidnekachew Dabessa,89.00,metad,"March 26th, 2015"
4,659651,Arabica,Brazil,Ji-Ae Ahn,88.83,scia,"September 3rd, 2013"


In [25]:
for i in len(df):
    url='https://database.coffeeinstitute.org/coffee/'+df['ID'][i]

https://database.coffeeinstitute.org/coffee/512197


### Grab the next level of pages

In [363]:
driver.get('https://database.coffeeinstitute.org/coffee/512197')
soup = BeautifulSoup(driver.page_source, 'html5lib')
time.sleep(1)
one_coffee=pd.read_html(driver.page_source)
len(one_coffee)

5

In [346]:
# Table 1 - I only want the total score
d = {'ID': [all_coffee['ID'][0]], 'Total_Score': [float(one_coffee[0][0][0])]}
Total_Score = pd.DataFrame(data=d)
Total_Score

,ID,Total_Score
0,512197,90.58


In [337]:
# Table 2
info1=one_coffee[1][[0, 1]]
info2=one_coffee[1][[2, 3]]
info2.rename(columns={2:0, 3:1}, inplace=True)
info0=pd.concat([info1, info2],  axis=0)
info=info0.set_index(0).T
info['ID']=all_coffee['ID'][0]
# info.set_index('ID', inplace=True)
info

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,Country of Origin,Farm Name,Lot Number,Mill,ICO Number,Company,Altitude,Region,Producer,Number of Bags,Bag Weight,In-Country Partner,Harvest Year,Grading Date,Owner,Variety,Status,Processing Method,ID
1,Ethiopia,METAD PLC,NaN,METAD PLC,2014/2015,METAD Agricultural Developmet plc,1950-2200,GUJI-HAMBELA/GOYO,METAD PLC,300,60 kg,METAD Agricultural Development plc,2014,"April 4th, 2015",metad plc,NaN,Completed,Washed / Wet,512197


In [338]:
# Table 3
cupping1=one_coffee[2][[0, 1]]
cupping2=one_coffee[2][[2, 3]]
cupping2.rename(columns={2:0, 3:1}, inplace=True)
cupping0=pd.concat([cupping1, cupping2],  axis=0)
cupping0
cupping0=cupping0.loc[cupping0[0] .isin(["Acidity", "Sweetness"])]
cupping=cupping0.set_index(0).T
cupping['ID']=all_coffee['ID'][0]
# cupping.set_index('ID', inplace=True)
cupping

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,Acidity,Sweetness,ID
1,8.75,10.00,512197


In [339]:
# Table 4
green1=one_coffee[3][[0, 1]]
green2=one_coffee[3][[2, 3]]
green2.rename(columns={2:0, 3:1}, inplace=True)
green0=pd.concat([green1, green2],  axis=0).set_index(0)
green0.drop([None], inplace=True)
green=green0.T
green['ID']=all_coffee['ID'][0]
# green.set_index('ID', inplace=True)
green

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,Moisture,Category One Defects,Quakers,Color,Category Two Defects,ID
1,12 %,0 full defects,0,Green,0 full defects,512197


In [351]:
# Put everything together
# Total_Score
# info
# cupping
# green
merge1=pd.merge(Total_Score, info, how="outer", on="ID")
merge2=pd.merge(merge1, cupping, how="outer", on="ID")
merge3=pd.merge(merge2, green, how="outer", on="ID")
merge3

,ID,Total_Score,Country of Origin,Farm Name,Lot Number,Mill,ICO Number,Company,Altitude,Region,...,Variety,Status,Processing Method,Acidity,Sweetness,Moisture,Category One Defects,Quakers,Color,Category Two Defects
0,512197,90.58,Ethiopia,METAD PLC,NaN,METAD PLC,2014/2015,METAD Agricultural Developmet plc,1950-2200,GUJI-HAMBELA/GOYO,...,NaN,Completed,Washed / Wet,8.75,10.00,12 %,0 full defects,0,Green,0 full defects


__Put everything together as one Function__

In [358]:
def get_one_coffee(i, one_coffee):
    # Table 1 - Total Score
    d = {'ID': [all_coffee['ID'][i]], 'Total_Score': [float(one_coffee[0][0][0])]}
    Total_Score = pd.DataFrame(data=d)
    Total_Score
    
    # Table 2 - Info
    info1=one_coffee[1][[0, 1]]
    info2=one_coffee[1][[2, 3]]
    info2.rename(columns={2:0, 3:1}, inplace=True)
    info0=pd.concat([info1, info2],  axis=0)
    info=info0.set_index(0).T
    info['ID']=all_coffee['ID'][i]

    # Table 3 - Cupping
    cupping1=one_coffee[2][[0, 1]]
    cupping2=one_coffee[2][[2, 3]]
    cupping2.rename(columns={2:0, 3:1}, inplace=True)
    cupping0=pd.concat([cupping1, cupping2],  axis=0)
    cupping0=cupping0.loc[cupping0[0] .isin(["Acidity", "Sweetness"])]
    cupping=cupping0.set_index(0).T
    cupping['ID']=all_coffee['ID'][i]
    
    # Table 4 - Green
    green1=one_coffee[3][[0, 1]]
    green2=one_coffee[3][[2, 3]]
    green2.rename(columns={2:0, 3:1}, inplace=True)
    green0=pd.concat([green1, green2],  axis=0).set_index(0)
    green0.drop([None], inplace=True)
    green=green0.T
    green['ID']=all_coffee['ID'][i]
    
    merge1=pd.merge(Total_Score, info, how="outer", on="ID")
    merge2=pd.merge(merge1, cupping, how="outer", on="ID")
    merge3=pd.merge(merge2, green, how="outer", on="ID")
    return merge3

### Loop over the list of all_coffee to get all the data

In [385]:
coffee_master=pd.DataFrame()

for i in range(len(all_coffee)):
    # Get the soup for one coffee
    url='https://database.coffeeinstitute.org/coffee/'+all_coffee['ID'][i]
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html5lib')
    one_coffee=pd.read_html(driver.page_source)

    # parse the data
    coffee1=get_one_coffee(i, one_coffee)
    
    # Add to master dataframe
    coffee_master=pd.concat([coffee_master, coffee1], axis=0)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [386]:
coffee_master

,ID,Total_Score,Country of Origin,Farm Name,Lot Number,Mill,ICO Number,Company,Altitude,Region,...,Variety,Status,Processing Method,Acidity,Sweetness,Moisture,Category One Defects,Quakers,Color,Category Two Defects
0,512197,90.58,Ethiopia,METAD PLC,NaN,METAD PLC,2014/2015,METAD Agricultural Developmet plc,1950-2200,GUJI-HAMBELA/GOYO,...,NaN,Completed,Washed / Wet,8.75,10.00,12 %,0 full defects,0,Green,0 full defects
0,398980,89.92,Ethiopia,METAD PLC,NaN,METAD PLC,2014/2015,METAD Agricultural Developmet plc,1950-2200,GUJI-HAMBELA/ALAKA,...,Other,Completed,Washed / Wet,8.58,10.00,12 %,0 full defects,0,Green,1 full defects
0,411500,89.75,Guatemala,"San Marcos Barrancas ""San Cristobal Cuch",NaN,NaN,NaN,NaN,1600 - 1800 m,NaN,...,Bourbon,Completed,NaN,8.42,10.00,0 %,0 full defects,0,NaN,0 full defects
0,632909,89.00,Ethiopia,Yidnekachew Dabessa Coffee Plantation,NaN,Wolensu,NaN,Yidnekachew Debessa Coffee Plantation,1800-2200,Oromia,...,NaN,Completed,Natural / Dry,8.42,10.00,11 %,0 full defects,0,Green,2 full defects
0,659651,88.83,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Completed,Natural / Dry,8.50,10.00,11 %,0 full defects,0,Bluish-Green,1 full defects
0,756923,88.83,Ethiopia,METAD PLC,NaN,METAD PLC,2014/2015,METAD Agricultural Developmet plc,1950-2200,GUJI-HAMBELA/BISHAN FUGU,...,Other,Completed,Washed / Wet,8.50,10.00,12 %,0 full defects,0,Green,2 full defects
0,372672,88.75,Peru,n/a,NaN,HVC,NaN,Richmond Investment-coffee department,NaN,NaN,...,Other,Completed,Washed / Wet,8.50,10.00,11 %,0 full defects,0,Bluish-Green,0 full defects
0,729069,88.67,Ethiopia,Aolme,NaN,C.P.W.E,010/0338,NaN,1570-1700,Oromia,...,NaN,Completed,NaN,8.42,9.33,3 %,0 full defects,0,NaN,0 full defects
0,112761,88.42,Ethiopia,Aolme,NaN,C.P.W.E,010/0338,NaN,1570-1700,Oromiya,...,NaN,Completed,NaN,8.42,9.33,3 %,0 full defects,0,NaN,0 full defects
0,523258,88.25,Ethiopia,Tulla Coffee Farm,NaN,Tulla Coffee Farm,2014/15,DIAMOND ENTERPRISE PLC,1795-1850,"SNNP/Kaffa Zone,GimboWereda",...,Other,Completed,Natural / Dry,8.50,10.00,10 %,0 full defects,0,Green,4 full defects


In [382]:
try: 
    x=1+'cat'
except(TypeError,ConnectionResetError):
    print('don\'t do that!')
    

don't do that!


### Grab the entire list of coffees from 1st page

In [418]:
all_coffee=pd.DataFrame()

In [428]:
driver.get('https://database.coffeeinstitute.org/coffees/arabica')

In [429]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [430]:
one_list=soup.find('table', {'data-table':'coffees'})

In [431]:
# This is just grabbing one page, will need to grab the rest of the pages later.
one_list = pd.read_html(driver.page_source)
one_list = pd.DataFrame(one_list[0])
one_list.drop(['Unnamed: 0'], axis = 1, inplace = True)
one_list['ID']=one_list['ID'].str.strip('#')
all_coffee=pd.concat([all_coffee, one_list], axis=0)

In [427]:
# "//" - anywhere in the document
# "a" - look for a tag
# "text()" look in the text part for 'Next'
next_button = driver.find_element_by_xpath("//a[contains(text(), 'Next')]")
next_button.click()

In [440]:
all_coffee

Arabica    100
Name: Species, dtype: int64

In [458]:
driver.get('https://database.coffeeinstitute.org/coffees/arabica')
all_coffee=pd.DataFrame()

for page in range(1,28):
    soup = BeautifulSoup(driver.page_source, 'html5lib')
    time.sleep(3)
    one_list=soup.find('table', {'data-table':'coffees'})
    time.sleep(3)    
    one_list = pd.read_html(driver.page_source)
    time.sleep(3)
    one_list = pd.DataFrame(one_list[0])
    one_list.drop(['Unnamed: 0'], axis = 1, inplace = True)
    one_list['ID']=one_list['ID'].str.strip('#')
    all_coffee=pd.concat([all_coffee, one_list], axis=0)
    
    next_button = driver.find_element_by_xpath("//a[contains(text(), 'Next')]")
    next_button.click()

print(all_coffee.shape)
all_coffee.sort_values(by=["ID"], inplace=True)
all_coffee[:10]

(100, 7)


,ID,Species,Country,Owner,Grade,ICP,Completed
36,112383,Arabica,Costa Rica,Francisco A Mena,86.33,scacr,"December 3rd, 2015"
33,112464,Arabica,Costa Rica,The Coffee Source Inc.,85.33,scaa,"May 1st, 2014"
8,112761,Arabica,Ethiopia,Ethiopia Commodity Exchange,88.42,ecx,"September 2nd, 2010"
27,13514,Arabica,Ethiopia,Assefa Belay Coffee Producer,85.42,metad,"March 31st, 2015"
18,138928,Arabica,United States (Hawaii),Kona Pacific Farmers Cooperative,85.58,scaa,"April 6th, 2012"
9,162715,Arabica,Guatemala,Juan Luis Alvarado Romero,85.83,anacafe,"June 5th, 2012"
23,167638,Arabica,Costa Rica,The Coffee Source Inc.,87.17,scaa,"April 2nd, 2014"
27,184738,Arabica,Brazil,NUCOFFEE,86.92,nucoffee,"December 3rd, 2011"
22,199122,Arabica,Ethiopia,Assefa Belay Coffee Producer,85.50,metad,"March 31st, 2015"
11,23456,Arabica,Guatemala,Juan Luis Alvarado Romero,85.83,anacafe,"July 3rd, 2012"


,ID,Species,Country,Owner,Grade,ICP,Completed
36,112383,Arabica,Costa Rica,Francisco A Mena,86.33,scacr,"December 3rd, 2015"
36,112383,Arabica,Costa Rica,Francisco A Mena,86.33,scacr,"December 3rd, 2015"
8,112761,Arabica,Ethiopia,Ethiopia Commodity Exchange,88.42,ecx,"September 2nd, 2010"
8,112761,Arabica,Ethiopia,Ethiopia Commodity Exchange,88.42,ecx,"September 2nd, 2010"
23,167638,Arabica,Costa Rica,The Coffee Source Inc.,87.17,scaa,"April 2nd, 2014"
23,167638,Arabica,Costa Rica,The Coffee Source Inc.,87.17,scaa,"April 2nd, 2014"
27,184738,Arabica,Brazil,NUCOFFEE,86.92,nucoffee,"December 3rd, 2011"
27,184738,Arabica,Brazil,NUCOFFEE,86.92,nucoffee,"December 3rd, 2011"
47,247690,Arabica,Brazil,NUCOFFEE,86.08,nucoffee,"December 3rd, 2011"
47,247690,Arabica,Brazil,NUCOFFEE,86.08,nucoffee,"December 3rd, 2011"
